## GPT-2（2019年）

GPT（Generative Pre-trained Transformer）是由 OpenAI 开发的一系列语言模型，其名称来源于模型的生成性质以及使用的 Transformer 架构。GPT 模型以其在生成文本方面的能力而闻名，能够生成连贯、语法正确且语义合理的文本。GPT-2（Generative Pre-trained Transformer 2）是由 OpenAI 于 2019 年推出的大型语言模型，它是 GPT 系列的第二代模型。GPT-2 继承了 GPT-1 的基本架构，但进行了显著的改进和扩展.

GPT-2 在一个更大的数据集 WebText 上进行训练，该数据集大约包含 40 GB 的文本数据和 800 万个文档。GPT-2 的训练方法与 GPT-1 相同，但使用了更大的模型架构和更强大的计算资源。GPT-2 拥有约 15 亿个参数，相比于 GPT-1 的参数数量有了显著增加，这使得它能够捕捉更复杂的语言模式。

GPT-2 是使用「transformer 解码器模块」构建的，而 BERT 则是通过「transformer 编码器」模块构建的。GPT-2与BERT的一个关键的不同在于：GPT-2 像传统的语言模型一样，一次只输出一个单词（token），而 BERT 则是一次性输出整个句子。这种模型之所以效果好是因为在每个新单词产生后，该单词就被添加在之前生成的单词序列后面，这个序列会成为模型下一步的新输入。这种机制叫做**自回归**（auto-regression）。GPT 模型是自回归的，这意味着它在生成文本时，每个新的词都是基于之前生成的词序列来预测的。BERT虽然没有使用自回归机制，但它获得了结合单词前后的上下文信息的能力，从而取得了更好的效果。

能够清楚地区分 BERT 使用的自注意力（self-attention）模块和 GPT-2 使用的带掩模的自注意力（masked self-attention）模块很重要。普通的自注意力模块允许一个位置看到它右侧单词的信息，而带掩模的自注意力模块则不允许这么做，只能看到左侧单词的信息。这使得 GPT-2 模型能够生成连贯、语法正确且语义合理的文本，而 BERT 模型则可以捕捉到更多的上下文信息。

<img src="./images/GPT/GPT2-1.png" style="zoom:60%;" />

GPT-2 可以处理最长 1024 个单词的序列。每个单词都会和它的前续路径一起「流过」所有的解码器模块。想要运行一个预训练的 GPT-2 模型，最简单的方法就是让它自己随机工作。我们也可以给它一点提示，让它生成一些关于特定主题的语句。在随机情况下，我们只简单地提供一个预先定义好的起始单词，让它自己生成文字。

<img src="./images/GPT/GPT2-2.png" style="zoom:60%;" />

​	此时，模型的输入只有一个单词，所以只有这个单词的路径是活跃的。单词经过层层处理，最终得到一个向量。向量可以对于词汇表的每个单词计算一个概率（GPT-2 的词汇表中有 50000 个单词）。

​	但有时这样会出问题——就像如果我们持续点击输入法推荐单词的第一个，它可能会陷入推荐同一个词的循环中，只有你点击第二或第三个推荐词，才能跳出这种循环。同样的，GPT-2 也有一个叫做「top-k」的参数，模型会从概率前 k 大的单词中抽样选取下一个单词。显然，在之前的情况下，top-k = 1。

<img src="./images/GPT/GPT2-3.GIF" style="zoom:100%;" />

​	接下来，我们将输出的单词添加在输入序列的尾部构建新的输入序列，让模型进行下一步的预测：

<img src="./images/GPT/GPT2-4.png" style="zoom:60%;" />

​	请注意，第二个单词的路径是当前唯一活跃的路径了。GPT-2 的每一层都保留了它们对第一个单词的解释，并且将运用这些信息处理第二个单词，GPT-2 不会根据当前的单词重新解释前面的单词。

### 输入编码

​	GPT-2 从嵌入矩阵中查找单词对应的嵌入向量，该矩阵也是模型训练结果的一部分。

<img src="./images/GPT/GPT2-5.png" style="zoom:60%;" />

​	每一行都是一个词嵌入向量：一个能够表征某个单词，并捕获其意义的数字列表。嵌入向量的长度和 GPT-2 模型的大小有关，最小的模型使用了长为 768 的嵌入向量来表征一个单词。

​	所以在一开始，我们需要在嵌入矩阵中查找起始单词对应的嵌入向量。但在将其输入给模型之前，我们还需要引入位置编码——一些向 transformer 模块指出序列中的单词顺序的信号。1024 个输入序列位置中的每一个都对应一个位置编码，这些编码组成的矩阵也是训练模型的一部分。

<img src="./images/GPT/GPT2-6.png" style="zoom:60%;" />

​	至此，输入单词在进入模型第一个 transformer 模块之前所有的处理步骤就结束了。如上文所述，训练后的 GPT-2 模型包含两个权值矩阵：嵌入矩阵和位置编码矩阵。

<img src="./images/GPT/GPT2-7.png" style="zoom:60%;" />

​	将单词输入第一个 transformer 模块之前需要查到它对应的嵌入向量，再加上 1 号位置位置对应的位置向量。


​	第一个 transformer 模块处理单词的步骤如下：首先通过自注意力层处理，接着将其传递给神经网络层。第一个 transformer 模块处理完但此后，会将结果向量被传入堆栈中的下一个 transformer 模块，继续进行计算。每一个 transformer 模块的处理方式都是一样的，但每个模块都会维护自己的自注意力层和神经网络层中的权重。

<img src="./images/GPT/GPT2-8.png" style="zoom:60%;" />

### 自注意力机制

​	语言的含义是极度依赖上下文的，比如下面这个机器人第二法则：

> 机器人第二法则
>
> 机器人必须遵守人类给它的命令，除非该命令违背了第一法则。

​	我在这句话中高亮表示了三个地方，这三处单词指代的是其它单词。除非我们知道这些词指代的上下文联系起来，否则根本不可能理解或处理这些词语的意思。当模型处理这句话的时候，它必须知道：

- 「它」指代机器人
- 「命令」指代前半句话中人类给机器人下的命令，即「人类给它的命令」
- 「第一法则」指机器人第一法则的完整内容

​	这就是自注意力机制所做的工作，它在处理每个单词（将其传入神经网络）之前，融入了模型对于用来解释某个单词的上下文的相关单词的理解。具体做法是，给序列中每一个单词都赋予一个相关度得分，之后对他们的向量表征求和。

​	举个例子，最上层的 transformer 模块在处理单词「it」的时候会关注「a robot」，所以「a」、「robot」、「it」这三个单词与其得分相乘加权求和后的特征向量会被送入之后的神经网络层。

<img src="./images/GPT/GPT2-9.png" style="zoom:60%;" />

​	自注意力机制沿着序列中每一个单词的路径进行处理，主要由 3 个向量组成：

1. 查询向量（Query 向量）：当前单词的查询向量被用来和其它单词的键向量相乘，从而得到其它词相对于当前词的注意力得分。我们只关心目前正在处理的单词的查询向量。
2. 键向量（Key 向量）：键向量就像是序列中每个单词的标签，它使我们搜索相关单词时用来匹配的对象。
3. 值向量（Value 向量）：值向量是单词真正的表征，当我们算出注意力得分后，使用值向量进行加权求和得到能代表当前位置上下文的向量。

<img src="./images/GPT/GPT2-10.png" style="zoom:60%;" />

​	一个简单粗暴的比喻是在档案柜中找文件。查询向量就像一张便利贴，上面写着你正在研究的课题。键向量像是档案柜中文件夹上贴的标签。当你找到和便利贴上所写相匹配的文件夹时，拿出它，文件夹里的东西便是值向量。只不过我们最后找的并不是单一的值向量，而是很多文件夹值向量的混合。

​	将单词的查询向量分别乘以每个文件夹的键向量，得到各个文件夹对应的注意力得分（这里的乘指的是向量点乘，乘积会通过 softmax 函数处理）。

<img src="./images/GPT/GPT2-11.png" style="zoom:60%;" />

​	我们将每个文件夹的值向量乘以其对应的注意力得分，然后求和，得到最终自注意力层的输出。

<img src="./images/GPT/GPT2-12.png" style="zoom:60%;" />

​	这样将值向量加权混合得到的结果是一个向量，它将其 50% 的「注意力」放在了单词「robot」上，30% 的注意力放在了「a」上，还有 19% 的注意力放在「it」上。我们之后还会更详细地讲解自注意力机制，让我们先继续向前探索 transformer 堆栈，看看模型的输出。

### 模型输出

​	当最后一个 transformer 模块产生输出之后（即经过了它自注意力层和神经网络层的处理），模型会将输出的向量乘上嵌入矩阵。

<img src="./images/GPT/GPT2-13.png" style="zoom:60%;" />

​	我们知道，嵌入矩阵的每一行都对应模型的词汇表中一个单词的嵌入向量。所以这个乘法操作得到的结果就是词汇表中每个单词对应的注意力得分。

<img src="./images/GPT/GPT2-14.png" style="zoom:60%;" />

​	我们简单地选取得分最高的单词作为输出结果（即 top-k = 1）。但其实如果模型考虑其他候选单词的话，效果通常会更好。所以，一个更好的策略是对于词汇表中得分较高的一部分单词，将它们的得分作为概率从整个单词列表中进行抽样（得分越高的单词越容易被选中）。通常一个折中的方法是，将 top-k 设为 40，这样模型会考虑注意力得分排名前 40 位的单词。

<img src="./images/GPT/GPT2-15.png" style="zoom:60%;" />

​	这样，模型就完成了一轮迭代，输出了一个单词。模型会接着不断迭代，直到生成一个完整的序列——序列达到 1024 的长度上限或序列中产生了一个终止符。

****

In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加载模型和分词器
model_name = './weights/gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 准备输入
prompt = "what is your name?"
inputs = tokenizer.encode(prompt, return_tensors='pt')

# 生成文本
output_sequences = model.generate(
    inputs,
    max_length=50,
    temperature=0.5,  # 控制生成文本的随机性
    num_return_sequences=1  # 生成文本的数量
)

# 解码生成的文本
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


what is your name?

I'm a little bit of a mystery. I'm not sure if I'm a real person or not. I'm just a guy who's been around the block for a while. I'm a guy who's


In [15]:
from transformers import GPT2Tokenizer, GPT2Model  # 导入GPT-2的分词器和模型类

# 从本地预训练权重初始化分词器
tokenizer = GPT2Tokenizer.from_pretrained('./weights/gpt2')
# 从本地预训练权重初始化GPT-2模型
model = GPT2Model.from_pretrained('./weights/gpt2')

# 定义要处理的文本
text = "what is your name?"
# 使用分词器对文本进行编码，并将结果转换为PyTorch张量
encoded_input = tokenizer(text, return_tensors='pt')
# 将编码后的输入传递给模型，并获取模型的输出
output = model(**encoded_input)
# 打印模型的输出结果
print(output)

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.2180,  0.0902, -0.3867,  ..., -0.1248, -0.0692, -0.2827],
         [-0.0182,  0.1850, -0.2534,  ..., -0.3534,  0.2311,  0.4750],
         [ 0.1262, -0.0620, -0.4503,  ..., -0.4268,  0.0285, -0.3058],
         [-0.5165,  0.2807, -1.2934,  ...,  0.1307, -0.0591,  0.4010],
         [-0.1379, -0.3302,  0.1139,  ...,  0.3398,  0.0379,  0.2840]]],
       grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-1.4767e+00,  2.0135e+00,  1.1144e+00,  ..., -1.1617e+00,
           -3.0150e-01,  1.6165e+00],
          [-1.8348e+00,  2.4955e+00,  1.7497e+00,  ..., -1.5397e+00,
           -2.3685e+00,  2.4482e+00],
          [-2.0957e+00,  3.2016e+00,  2.2118e+00,  ..., -1.7556e+00,
           -1.7967e+00,  2.1215e+00],
          [-3.0464e+00,  2.5299e+00,  1.4454e+00,  ..., -8.6774e-01,
           -9.5575e-01,  1.5480e+00],
          [-2.5361e+00,  2.9027e+00,  1.9364e+00,  ..., -1.0628e+00,
           -2.3543e+00,  1.9235e+00